In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import mibitrans as mbt


In [ ]:
ft = 3.281
hydro = mbt.HydrologicalParameters(
    velocity=350/ft/365,
    h_gradient=0.048,        # Hydraulic gradient [-]
    h_conductivity=0.495,    # Hydraulic conductivity [m/day]
    porosity=0.25,           # Effective soil porosity [-]
    alpha_x=13.3/ft,         # Longitudinal dispersivity, in [m]
    alpha_y=1.3/ft,          # Transverse horizontal dispersivity, in [m]
    alpha_z=0                # Transverse vertical dispersivity, in [m]
)

att = mbt.AttenuationParameters(
    # Soil bulk density in [g/m^3]
    bulk_density=1.7,
    # Partition coefficient of the transported contaminant to soil organic matter, in [m^3/g]
    partition_coefficient=38,
    # Fraction of organic material in the soil [-]
    fraction_organic_carbon=5.7e-5,
    # Molecular diffusion, in [m2/day]
    diffusion=0,
    # Contaminant half life, in [days]
    half_life=0
)

source = mbt.SourceParameters(
    source_zone_boundary=np.array([30/ft, 35/ft, 40/ft]),
    source_zone_concentration=np.array([13.68, 2.508, 0.057]),
    depth=10/ft,
    total_mass=25000
)

model = mbt.ModelParameters(
    # Model extent in the longitudinal (x) direction in [m].
    model_length = 1000/ft,
    # Model extent in the transverse horizontal (y) direction in [m].
    model_width = 300/ft,
    # Model duration in [days].
    model_time = 10 * 365,
    # Model grid discretization step size in the longitudinal (x) direction, in [m].
    dx = 5/ft,
    # Model grid discretization step size in the transverse horizontal (y) direction, in [m].
    dy = 2/ft,
    # Model time discretization step size, in [days]
    dt = 365/5
)

In [ ]:
class AnatransAlternativeSource(mbt.Anatrans):
    """Mibitrans model class with alternative calculations."""

    def __init__(
    self,
    hydrological_parameters,
    attenuation_parameters,
    source_parameters,
    model_parameters,
    verbose=False,
    ):
        """Initialize model class."""
        super().__init__(hydrological_parameters, attenuation_parameters, source_parameters, model_parameters, verbose)

    def _equation_term_source_depletion(self, xxx, ttt):
        term = np.exp(-self.k_source * (ttt - xxx / self.rv))
        # Term can be max 1; can not have 'generation' of solute ahead of advection.
        return np.where(term > 1, 1, term)

    def _equation_decay_sqrt(self):
        return np.sqrt(1 + 4 * self._decay_rate * self._hyd_pars.alpha_x / self.rv)

In [ ]:
ana_object = mbt.Anatrans(hydro, att, source, model)
ana_results = ana_object.run()

alt_object = AnatransAlternativeSource(hydro, att, source, model)
alt_results = alt_object.run()

mbt_object = mbt.Mibitrans(hydro, att, source, model, verbose=True)
mbt_results = mbt_object.run()


In [ ]:
%matplotlib ipympl

In [ ]:
plt.clf()
ana_results.centerline(color="green", label="anatrans")
alt_results.centerline(color="red", linestyle="--", label="anatrans alternative")
mbt_results.centerline(color="blue", linestyle=":", label="mibitrans")
plt.legend()
plt.show()

In [ ]:
plt.clf()
anim = mbt.centerline(model=[ana_results, alt_results, mbt_results],
                  legend_names=["Anatrans model", "Alternative model", "mibitrans model"],
                  linestyle=":",
                  animate=True)
plt.show()